In [1]:
import os
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import community.community_louvain as community_louvain
import pandas as pd

In [2]:
os.makedirs("img", exist_ok=True)

In [3]:
# ==============================
# 1. Data
# ==============================
G = nx.read_edgelist("facebook_combined.txt", nodetype=int)
print("Nodes:", G.number_of_nodes())
print("Edges:", G.number_of_edges())

# Layout pre vizualizáciu
pos = nx.spring_layout(G, seed=42)

Nodes: 4039
Edges: 88234


In [4]:
# ==============================
# 2. Vizualizácia pôvodných dát
# ==============================
plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G, pos, node_size=10)
nx.draw_networkx_edges(G, pos, alpha=0.05)
plt.title("Facebook social network – data")
plt.axis("off")
plt.savefig("img/data.png", dpi=300)
plt.close()

In [5]:
# ==============================
# 3. Modely detekcie komunít
# ==============================
# Louvain
partition_louvain = community_louvain.best_partition(G)

# LPA
communities_lpa = list(nx.algorithms.community.label_propagation_communities(G))
partition_lpa = {}
for idx, comm in enumerate(communities_lpa):
    for node in comm:
        partition_lpa[node] = idx

In [6]:
# ==============================
# 4. Funkcia na vizualizáciu a uloženie
# ==============================
def draw_and_save(graph, partition, title, filename, pos):
    plt.figure(figsize=(12, 12))
    communities = list(set(partition.values()))
    cmap = cm.get_cmap('tab20', len(communities))

    for i, comm in enumerate(communities):
        nodes = [n for n in graph.nodes() if partition[n] == comm]
        nx.draw_networkx_nodes(
            graph, pos,
            nodelist=nodes,
            node_size=20,
            node_color=[cmap(i)]
        )

    nx.draw_networkx_edges(graph, pos, alpha=0.03)
    plt.title(title)
    plt.axis("off")
    plt.savefig(filename, dpi=300)
    plt.close()

# ==============================
# 5. Uloženie vizualizácií
# ==============================
draw_and_save(G, partition_louvain, "Detekcia komunít – Louvain", "img/louvain.png", pos)
draw_and_save(G, partition_lpa, "Detekcia komunít – LPA", "img/lpa.png", pos)


/tmp/ipykernel_36689/464718339.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(communities))


In [7]:
# ==============================
# 6. Vyhodnotenie – modularita a počet komunít
# ==============================
mod_louvain = community_louvain.modularity(partition_louvain, G)
mod_lpa = community_louvain.modularity(partition_lpa, G)

print("Louvain – modularita:", mod_louvain, "| počet komunít:", len(set(partition_louvain.values())))
print("LPA     – modularita:", mod_lpa, "| počet komunít:", len(set(partition_lpa.values())))


Louvain – modularita: 0.8347388378412373 | počet komunít: 16
LPA     – modularita: 0.7368407345348218 | počet komunít: 44


In [8]:
# ==============================
# Uloženie tabuľky s výsledkami
# ==============================
mod_louvain_rounded = round(mod_louvain, 4)
mod_lpa_rounded = round(mod_lpa, 4)

results = {
    "Model": ["Louvain", "LPA"],
    "Počet komunít": [len(set(partition_louvain.values())), len(set(partition_lpa.values()))],
    "Modularita": [mod_louvain_rounded, mod_lpa_rounded]
}

df_results = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(6, 2))  # veľkosť obrázku
ax.axis('tight')
ax.axis('off')

# Vytvorenie tabuľky
table = ax.table(cellText=df_results.values,
                 colLabels=df_results.columns,
                 cellLoc='center',
                 loc='center')

table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 2)  # mierka riadkov a stĺpcov

# Uloženie do PNG
plt.savefig("img/vyhodnotenie.png", dpi=300, bbox_inches='tight')
plt.close()
